In [4]:
# Classes File

In [3]:
# Local Import
import numpy as np
# import pandas as pd
import math
import scipy.stats as ss
import numpy.linalg as la
from itertools import product
import numpy.random as nr
import pickle

In [6]:
# OBJECTS

In [12]:
''' Transform Range of Data into Unit Interval Space '''
def UI(x, b=True):
    ''' For Univariate and Aggregate Regression '''
    if b:
        mimi = np.min([i for j in x for i in j])
        mama = np.max([i for j in x for i in j])
        return (x - mimi) / (mama - mimi)
    else:
        return (x - np.min(x)) / (np.max(x) - np.min(x))
def UII(x, b=False):
    ''' For Multivariate Regression '''
    if b:
        mimi = np.asarray([np.min(i) for i in x.T])
        mama = np.asarray([np.max(i) for i in x.T])
    else:
        mm = np.asarray([i for j in x for i in j]).T
        mimi = np.asarray([np.min(i) for i in mm])
        mama = np.asarray([np.max(i) for i in mm])
    
    return [(i - mimi) / (mama - mimi) for i in x]

''' Converts subarrays into lists '''
def npl(x):
    return np.asarray([i.tolist() for i in x])

In [13]:
class Wrangle:
    ''' Data Wrangling Class for Test Data 
    '''
    def __init__(self, data, aggregate=False):
        ''' Numpy-nize Census Tract 
        '''
        if 'PUMA5' in data:
            data = data.drop(['PUMA5'], axis=1)
        self.Key = data.keys()
        if aggregate:
            Y = np.asarray(data.loc[:, self.Key[1]])
            if len(self.Key) > 3:
                X = np.asarray(data.loc[:,self.Key[2]:])
            else:
                X = np.asarray(data.loc[:, self.Key[2]])
            self.N = len(X)
        else:
            if len(self.Key) > 3:
                Cell = np.unique(np.array(data[self.Key[0]])).astype(int)
                Y = [np.asarray(data.loc[data.loc[:, self.Key[0]]== i, self.Key[1]]) for i in Cell]
                X = [np.asarray(data[data.Cell==i].loc[:,self.Key[2]:]) for i in Cell]
                self.N = np.asarray([len(i) for i in X])
            else:
                Cell = np.unique(np.array(data[self.Key[0]])).astype(int)
                Y = [np.asarray(data.loc[data.loc[:, self.Key[0]]== i, self.Key[1]]) for i in Cell]
                X = [np.asarray(data.loc[data.loc[:, self.Key[0]]== i, self.Key[2]]) for i in Cell]
                self.N = np.asarray([len(i) for i in X])
        self.X = X
        self.Y = Y 
        self.v = len(self.Key) - 1
                
    def __call__(self):
        return self.X, self.Y, self.N, self.v

In [14]:
class Methods:
    ''' Class of regression methods and their parts
    '''
    def __init__(self, X, Y, N, v, method=[]):
        ''' Make sure you are using at most 2-dimensional arrays for X, Y
        '''
        # We assume X is organised by index with dependent variables inside each
        self.X = np.asarray(X)
        self.Y = np.asarray(Y)
        self.method = method
        if "SMDM" in self.method:
            self.form = ro.Formula("y~x")
        self.N = N
        self.v = v
#         self.o = np.asarray([list(i) for i in product([0, 1], repeat = self.v)])
        self.o = np.asarray([list(i) for i in product([0, 1], repeat = self.v)])
        if type(self.N) is int:
            self.index = [None]
        else:
            self.index = range(len(self.N))
        self.range = range(len(self.o))
        self.nd75 = ss.norm.ppf(0.75)
        if "Winsorize" in self.method:
            self.win = True
        else:
            self.win = False
        if "SE" in self.method:
            self.SE = True
        else:
            self.SE = False
            
        # Maxiter-rer
        for i in self.method:
            try:
                maxiter = int(i)
            except:
                pass
        try:
            self.maxiter = maxiter
        except:
            self.maxiter = 100
    
    def sort(self, I=None, O=None, win=False):
        ''' Sort arrays and append outlier variations
        '''
        if I is None:
            y = self.Y
            x = self.X
        else:
            y = self.Y[I]
            x = self.X[I]
        
        if O is not None:
            y = np.block([y, self.o[O][0]])
            if self.v > 2 :
                x = np.vstack([x, self.o[O][1:]])
            else:
                x = np.block([x, self.o[O][1]])
        
        if win:
            x = ss.mstats.winsorize(x, limits=0.05)
            x = x.filled()
            y = ss.mstats.winsorize(y, limits=0.05)
            y = y.filled()
        
        return x, y
    
    def stack(self, x):
        ''' Add constant to estimate 
        '''
        return np.block([[np.ones(len(x))], [x.T]]).T
    
    def HCE(self, X, r, a):
        ''' Calculate Heteroscedasticity-consistent standard errors
        '''
        HCE = np.dot(np.dot(a, X.T.dot(np.dot(np.diag(np.square(r)), X))), a)
        return np.array(np.diag(HCE))
    
    def lstsq(self, x, y, w=None, se=False):
        ''' Calculus linear least-squares
        '''
        x = self.stack(x)
        if w is not None:
            X = x.T.dot(np.diag(w)).T
        else:
            X = x
        a = la.inv(X.T.dot(x))
        β̂ = np.dot(a, X.T.dot(y))
        r = y - np.dot(X, β̂)
        if se:
            se = self.HCE(X, r, a)
            #se = np.sqrt(r.dot(r) / np.sum(np.square(X.T[1] - np.mean(X.T[1]))) / (len(r) - self.v))
        return β̂, r, se

    # OLS Method:
    def OLS(self, I=None, O=None, se=False):
        x, y = self.sort(I=I, O=O, win=self.win)
        β̂, r, se = self.lstsq(x, y, None, se)
        return β̂, r, se

    # Most B-Robust Method:
    def Huber(self, I=None, O=None, se=False):
        x, y = self.sort(I=I, O=O, win=self.win)
        norm = la.norm(x)
        x = x / norm
        y = y / norm
        β̂, r, se = self.lstsq(x, y, None, se)
        return β̂, r, se
        
    # MM Method:
    def MAD(self, r):
        ''' Median Absolute Deviation 
        '''
        return np.median(np.absolute(r - np.median(r)))
    
    def Tukey(self, u, c=1.547):
        ''' Tukey's biweight function ''' 
        ρ = np.power(u, 2) / 2 - np.power(u, 4) / (2 * math.pow(c, 2)) + np.power(u, 6) / (6 * math.pow(c, 4))
        if np.any(np.abs(u) > c):
            ρ[np.abs(u) > c] = math.pow(c, 2) / 6            
        return ρ
            
    def σ(self, r, w=None, K=0.199):
        ''' The scale for the S-Estimator we wish to minimize
        '''
        if w is None:
            σ = self.MAD(r) / self.nd75 
        else:
            σ = np.sqrt(np.sum(np.multiply(np.square(r),w)) / (len(r) * K))
        return σ
    
    def u(self, r, σ):
        ''' Bisquare ratio 
        '''
        return r / σ
    
    def weight(self, u, method="S", it=False):
        ''' Estimator Reweighing
        '''
        if method=="S":
            if not it:
            # iteration = 1
                w = np.square(u / 1.547)
                if np.any(np.abs(u) > 1.547):
                    w[np.abs(u) > 1.547] = 0
                weight = np.square(1 - w)
            else:
                weight = self.Tukey(u) / np.square(u)
        if method=="MM":
            w = np.square(u / 4.685)
            if np.any(np.abs(u) > 4.685):
                w[np.abs(u) > 4.685] = 0
            weight = np.square(1 - w)
        return weight

    def MM(self, I=None, O=None, SE=True):
        ''' MM-Estimator regression algorithm as specified in Susanti 2009
        '''
        x, y = self.sort(I, O, win=self.win)
        β̂, r, se = self.lstsq(x, y, None, False)
        # 2. S-Estimation
        # 1st iteration
        σ = self.σ(r)
        u = self.u(r, σ)
        w = self.weight(u, "S", False)
        β̂, r, se = self.lstsq(x, y, w, False)
        # IRWLS
        for i in range(self.maxiter):
            σ = self.σ(r, w)
            u = self.u(r, σ)
            w = self.weight(u, "S", True)
            S, r, se = self.lstsq(x, y, w, False)
            if np.allclose(S, β̂, rtol=1e-09):
#             if math.isclose(la.norm(S - β̂), 0, abs_tol=1e-09):
                break
            else:
                β̂ = S
        # We use the MAD of the residuals of the S-estimator
#         σ = self.σ(r)
        # Slightly Worse accuracy but closer to OLS estimate
        σ = self.σ(r, w)
        # 3. MM-Estimate Convergence 
        for i in range(self.maxiter):
            u = self.u(r, σ)
            w = self.weight(u, "MM")
            M, r, se = self.lstsq(x, y, w, False)
#             if math.isclose(la.norm(M - β̂), 0, abs_tol=1e-03):
            if np.allclose(M, β̂, rtol=1e-04):
                break
            else:
                β̂ = M
        if SE:
            β̂s, rs, se = self.lstsq(x, y, w, SE)
        return M, r, se, σ   
        
    # SMDM Method:
    def SMDM(self, I=None, O=None):
        ''' Run SMDM from R's 'lmrob' package via rpy2
        '''
        x, y = self.sort(I, O, win=self.win)
        try:
            nr, nc = x.shape
        except:
            nr = x.shape[0]
            nc = 1
        # Set up formula environment
        rnp.activate()
        self.form.environment["y"] = ro.r['matrix'](y, nrow=nr, ncol=1)
        self.form.environment["x"] = ro.r['matrix'](x, nrow=nr, ncol=nc)
        # Run Robust Regression
        lmr = robustbase.lmrob(self.form, method = "SMDM", setting="KS2014")
        # must copy variables because of memory constraint
        betas = np.array(lmr.rx2("coefficients"), copy=True)
        scale = np.array(lmr.rx2("scale"), copy=True)
        residuals = np.array(lmr.rx2("residuals"), copy=True)
        stderr = np.array(np.diag(lmr.rx2("cov")), copy=True)
        return betas, residuals, stderr, scale
    
    def __call__(self, withoutliers=True):
        ''' Specify regression method and run all regression in one/two line(s)
        '''
        if "OLS" in self.method:
            if self.index is None:
                regression = self.OLS(None, None, self.SE)
                if withoutliers:
                    withoutliers = np.asarray([self.OLS(None, O, self.SE) for O in self.range])
            else:
                regression = np.asarray([self.OLS(I, None, self.SE) for I in self.index])
                if withoutliers:
                    withoutliers = np.asarray([[self.OLS(I, O, self.SE) for O in self.range] for I in self.index]) 
        if "H" in self.method:
            if self.index is None:
                regression = self.Huber(None, None, self.SE)
                if withoutliers:
                    withoutliers = np.asarray([self.Huber(None, O, self.SE) for O in self.range])
            else:
                regression = np.asarray([self.Huber(I, None, self.SE) for I in self.index])
                if withoutliers:
                    withoutliers = np.asarray([[self.Huber(I, O, self.SE) for O in self.range] for I in self.index])    
        elif "MM" in self.method:
            if self.index is None:
                regression = self.MM(None, None, self.SE)
                if withoutliers:
                    withoutliers = np.asarray([self.MM(None, O, self.SE) for O in self.range])
            else:
                regression = np.asarray([self.MM(I, None, self.SE) for I in self.index])
                if withoutliers:
                    withoutliers = np.asarray([[self.MM(I, O, self.SE) for O in self.range] for I in self.index])
        elif "SMDM" in self.method:
            if self.index is None:
                regression = self.SMDM(None, None)
                if withoutliers:
                    withoutliers = np.asarray([self.SMDM(None, O) for O in self.range])
            else:
                regression = np.asarray([self.SMDM(I, None) for I in self.index])
                if withoutliers:
                    withoutliers = np.asarray([[self.MM(I, O) for O in self.range] for I in self.index])
        return regression, withoutliers

In [15]:
class DWORK:
    def __init__(self, Tr, r, o, ϵ, psize, method=["Laplace", "BETA", "IQR"]):
        ''' Dwork & Lei Release Algorithm
            Developed for releasing true β w/ noise
            Can be used for releasing SE's as well
        '''
        self.method = method
        # length of each cell
        self.n = psize
        # base for each cell (for S algorithm)
        self.base = np.asarray([1 + 1 / np.log(i) for i in self.n])
        
        # store β̂'s and β̂ᵢ's:
        if "BETA" in self.method:
            self.Tb = Tr[0][0]
            self.b = npl(r.T[0]).T
            self.db = npl(o.T[0].T)
        elif "SE" in self.method:
            self.Tb = Tr[0][2]
            self.b = npl(r.T[2]).T
            self.db = npl(o.T[2].T)
        if "MAD" in self.method:
            self.residuals = npl(r.T[1]).T
            self.dresiduals = npl(o.T[1].T)
            self.nd75 = ss.norm.ppf(0.75)

        # number of partitions & range
        self.pee = np.sum(self.n) / self.n
        self.p = len(self.n)
        self.range = range(self.p)
        # total sample size
        self.N = np.sum(self.n)
        self.ϵ = ϵ
        # define delta parameter
        ndivp = self.N / self.p
        self.δ = 0.5 * math.pow(ndivp, (- ϵ) * math.log(ndivp))
        self.RANGE = range(len(self.db[0]))
            
        if "MAD" in self.method:
            self.VRANGE = []
            self.vars = len(self.b.T[0])
        else:
            self.VRANGE = range(len(self.b.T[0]))
    
    def ω(self, h=1, method=["Laplace"]):
        ''' Laplace of Gaussian Mechanism
        '''
        if "Laplace" in method:
            return nr.laplace(0, h/self.ϵ)
        elif "Gaussian" in method:
            return nr.normal(0, h * math.sqrt(2 * math.log(1.25 / self.δ)) / self.ϵ)
        else:
            raise ValueError('No Mechanism Specified')
    
    # 2. Run S algorithm
    def IQR(self, b, db=None):
        ''' Function: Interquartile Range of partitioned β̂'s
        '''
        if db is not None:
            # Array like O array but with iqr calc for switching old β with new β
            nb = np.asarray([[np.block([[np.delete(b.T, i, 0)], [db[i][j]]]) for j in self.RANGE] for i in self.range])
            IQR = np.asarray([[[np.percentile(k, 75) - np.percentile(k, 25) for k in i.T] for i in j]for j in nb])
        else:
            # Original IQR for β
            IQR = np.asarray([np.percentile(i, 75) - np.percentile(i, 25) for i in b])
        return IQR
    
    def mad(self, r):
        ''' Median Absolute Deviation 
        '''
        return np.median(np.absolute(r - np.median(r))) / self.nd75

    def BMAD(self, b, db=None):
        ''' NEW: MAD for Beta's instead of residuals
        '''
        if db is not None:
            # Array like O array but with iqr calc for switching old β with new β
            nb = np.asarray([[np.block([[np.delete(b.T, i, 0)], [db[i][j]]]) for j in self.RANGE] for i in self.range])
            BMAD = np.asarray([[[mad(k) for k in i.T] for i in j]for j in nb])
        else:
            # Original IQR for β
            BMAD = np.asarray([mad(i) for i in b])
        return BMAD
    
    def MAD(self, r, dr=None):
        ''' NEW: We calculate the MAD for the residuals of each β̂; instead of the IQR
        '''
        if dr is not None:
            # Array like O array but with iqr calc for switching old β with new β
            nb = []
            for i in self.range:
                ab = []
                for j in self.RANGE:
                    K = r.tolist()
                    K[i] = dr[i][j]
                    ab.append(K)
                nb.append(ab)
            nb = np.asarray(nb)
            # Returns Median Absolute Deviation for each altered partition
            MAD = np.asarray([[[self.mad(k) for k in i.T] for i in j]for j in nb])
            # Returns IQR of MAD - n.b.: you have instead chosen to go for the norm
#             IQR = np.asarray([[np.percentile(j, 75) - np.percentile(j, 25) for j in i] for i in MAD])
            GMAD = np.asarray([[la.norm(j) for j in i] for i in MAD])
            
        else:
            # Original MAD for β
            # Returns Median Absolute Deviation for each partition
            MAD = np.asarray([self.mad(i) for i in r])
#             IQR = np.asarray(np.percentile(MAD, 75) - np.percentile(MAD, 25))
            GMAD = np.asarray(la.norm(MAD))
        return GMAD
        
    
    def H(self, o=False):
        ''' Part 1: H
            - Compute H for each β̂ and compute H' for alt β̂'s'
        '''
        if o:
            if "MAD" in self.method:
                SCALE = self.MAD(self.residuals, self.dresiduals)
                H = np.asarray([np.log(SCALE[i]) / np.log(self.base[i]) for i in self.range])
            else:
                if "IQR" in self.method:
                    SCALE = self.IQR(self.b, self.db)
                elif "BMAD" in self.method:
                    SCALE = self.BMAD(self.b, self.db)
                H = np.asarray([[np.log(SCALE[i][j]) / np.log(self.base[i]) for j in self.RANGE]for i in self.range])                
        else:
            if "MAD" in self.method:
                SCALE = self.MAD(self.residuals)
            else:
                if "IQR" in self.method:
                    SCALE = self.IQR(self.b)  
                elif "BMAD" in self.method:
                    SCALE = self.BMAD(self.b)  
            H = np.asarray([np.log(SCALE) / np.log(self.base[i]) for i in self.range])
        return H
    
    def S(self, BOOL=False):
        ''' Part 2: S Algorithm
            - Compute Noise-infused SCALE for β̂'s
        '''
        # Log IQR / Log 1 + 1/n
        H = self.H()
        dH = self.H(True)
        
        # 2.3 - Bins for each H
        bins = np.asarray([[np.abs(H[i] - dH[i][j]) for j in self.RANGE] for i in self.range])
        
        # 2.4 return TRUE for violation of 2.3 <= 1
        if "MAD" in self.method:
            booL = np.asarray([np.any(bins[i] > 1) for i in self.range])
            s = np.asarray([self.MAD(self.residuals) * (self.base[i] ** abs(self.ω())) for i in self.range])
            s[booL] = np.nan            
        else:
            booL = np.asarray([[np.any(bins[i].T[j] > 1) for j in self.VRANGE] for i in self.range])
            if "IQR" in self.method:
                s = np.asarray([self.IQR(self.b) * (self.base[i] ** abs(self.ω())) for i in self.range])
            elif "BMAD" in self.method:
                s = np.asarray([self.BMAD(self.b) * (self.base[i] ** abs(self.ω())) for i in self.range])
            s[booL] = np.nan

        if BOOL:
            s = booL
        return s

    # 3. compute h for each s
    def h(self, s):
        h = np.asarray([s[i] / math.pow(self.N, 1 / (2 * self.pee[i])) for i in self.range])
        return h
    
    def z(self, h):
        if "MAD" in self.method:
            stackh = np.ones((self.vars, self.p)) * h
            return np.asarray([[self.ω(j, self.method) for j in i] for i in stackh.T])
        else:
            return np.asarray([[self.ω(j, self.method) for j in i] for i in h])        
    
    def RH(self, s, BOOL=False):
        ''' Part 3: Release
            - Compute True β + Noise
        '''
        h = self.h(s)
        # 3.1 np.abs(alt β̂'s - β̂) <= h array
        bins = np.asarray([[np.abs(self.b.T[i] - self.db[i][j]) for j in self.RANGE] for i in self.range])
        
        # return True for violation        
        if "MAD" in self.method:
            anybooL = np.asarray([np.any(bins[i] > h[i]) for i in self.range])
            boolie = [np.any(i) for i in anybooL]
        else:
            anybooL = np.asarray([[np.any(bins[i].T[j] > h[i][j]) for j in self.VRANGE] for i in self.range])
            boolie = [np.any(i) for i in anybooL]            
            
        # 3.2 for TRUE compute β + noise
        RHb = self.Tb + self.z(h)
        RHb[anybooL] = np.nan   
        if BOOL:
            return anybooL
        elif "Local" in self.method:
            return RHb
        else:
            # 3.3 find min(β + noise)
            similar = [la.norm(self.Tb - i) for i in RHb]
            try:
                index = np.nanargmin(similar)
                return RHb[index]
            except:
                return np.full(len(self.b.T[0]), np.nan)
            
    def __call__(self, BOOL=False):
        ''' Set up so one can return Boolean Arrays for both S and RH algorithm 
        ''' 
        s = self.S()
        RH = self.RH(s, BOOL)
        if BOOL:
            S = self.S(True)
        elif "Local" in self.method:
            if "MAD" in self.method:
                S = self.MAD(self.residuals) * ((1 + 1 / np.log(self.N)) ** self.ω())
            else:
                if "IQR" in self.method:
                    S = self.IQR(self.b) * ((1 + 1 / np.log(self.N)) ** self.ω())
                elif "BMAD" in self.method:
                    S = self.BMAD(self.b) * ((1 + 1 / np.log(self.N)) ** self.ω())
        else:
            S = s
            
        return S, RH

In [16]:
class MOSE:
    ''' Release Algorithm by Chetty & Friedman
    '''
    def __init__(self, regression, withoutliers, ϵ, N, method=["Laplace"]):
        ''' We assume we are working at the PUMS Code Level, not the Tract Level
        '''
        self.RA = regression
        self.RB = withoutliers
        self.range = range(4)
        if type(N) is int:
            self.N = 1
            self.L = self.N
            self.index = range(self.N)
        else:
            self.N = N
            self.L = len(self.N)
            self.index = range(self.L)
        self.method = method
        self.ϵ = ϵ
            
    def ω(self, size):
        ''' Draws from Laplace or Normal Distribution 
        '''
        if "Laplace" in self.method:
            return nr.laplace(0, 1/math.sqrt(2), size)
        elif "Gaussian" in self.method:
            return nr.normal(0, 1, size)
    
    def LS(self):
        ''' Calculate Local Sensitivity 
        '''
        return np.asarray([np.max(np.abs(npl(self.RB.T[0].T)[i].T[1] - npl(self.RA.T[0].T)[i][1])) for i in self.index])
    
    def mOSE(self):
        ''' Calculate Maximum Observed Local Sensitivity '''
        MOSE = np.max(self.N * self.LS())
        return np.asarray([npl(self.RA.T[i]).T for i in [0, 2]]), MOSE
    
    
    def noise(self, θseθ, χ):
        ''' Add Noise to Statistics 
        '''
        S = np.asarray(self.N).astype(float)
        noise = lambda x: x * math.sqrt(2) * self.ϵ
        
        nθ = np.asarray([i + χ * noise(self.ω(self.L)) / S for i in θseθ[0]])
        senθ = np.asarray(np.sqrt([np.square(i) + 2 * np.square(χ / S) for i in θseθ[1]])) 
        nsenθ = np.asarray([i + χ * noise(self.ω(self.L)) / S for i in senθ])
        nN = self.N * (1 + noise(self.ω(self.L)) / S)
        return nθ, nsenθ, nN
    
    def Ω(self, χ, N):
        ''' Draws from Laplace Distribution 
        '''
        scale = χ / (N * self.ϵ)
        return nr.laplace(0, scale, self.L)

    def correctnoise(self, θseθ, χ):
        ''' Add Noise to Statistics 
        '''
        S = np.asarray(self.N).astype(float)
        
        nθ = np.asarray([i + self.Ω(χ, S) for i in θseθ[0]])
        senθ = np.asarray(np.sqrt([np.square(i) + 2 * np.square(χ / S) for i in θseθ[1]])) 
        nsenθ = np.asarray([i + self.Ω(χ, S) for i in senθ])
        nN = self.N * (1 + self.Ω(χ, S) / χ)
        return nθ, nsenθ, nN
#         senθ = np.square(θseθ[1][1]) + 2 * np.square(χ / S) 
#         np.sqrt(sen.astype(float))
#         senθ + χ * noise(self.ω(self.L)) / S
#         nsenθ = np.asarray(["Sample Size Too Small" if i==np.inf else i for i in nsenθ])

    def __call__(self, LS=False):
        ''' Release Noise Infused Statistics 
        '''
        if LS:
            return self.LS()
        else:
            θandSEθ, mose = self.mOSE()
            if "Correct" in self.method:
                nθ, nsenθ, nN = self.correctnoise(θandSEθ, mose)
            else:
                nθ, nsenθ, nN = self.noise(θandSEθ, mose)
            return nθ.T, nsenθ.T, nN, self.ϵ, mose

In [22]:
class MSE:
    ''' We compute MSE's for the DWORK algorithm outputs for different epsilons
    '''
    def __init__(self, Tr, r, o, psize, method=["Laplace", "DWORK", "BETA"]):    
        self.T = Tr[0][0]
        self.Tr = Tr
#         self.To = To
        self.r = r
        self.o = o
        self.psize = psize
        self.method = method
        self.ϵ = [j/10 for j in range(1, 11)]
        if "MAD" in self.method:
            self.k = 2
        else:
            self.k = 10
        
    def __call__(self):
        MSE = []
        for ϵ in self.ϵ:
            diff = []
            # We repeat 10 times
            for j in range(self.k):
                if "DWORK" in self.method:
                    s, noise = DWORK(self.Tr, self.r, self.o, ϵ, self.psize, self.method)()
                elif "MOSE" in self.method:
                    noise, nsenθ, nN, ϵ, mose = MOSE(self.r, self.o, ϵ, self.psize, self.method)()
#                     index = np.nanargmin([la.norm(self.T - i) for i in nθ])
                try:
                    # Squared Error for Beta Vector
                    l2norm = np.nanmin([np.square(la.norm(self.T - i)) for i in noise])
                except:
                    l2norm = np.nan
#                 diff.append(np.square(np.subtract(noise, self.T)))
                diff.append(l2norm)
            # Compute Mean of Squared Error
            MSE.append(np.nanmean(diff))
        # Return MSE Array
        return MSE